In [5]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 227 kB in 3s (89.8 kB/s)
Reading package lists... Done


In [9]:
# Import packages
from pyspark.sql import SparkSession
import time
import requests
from io import StringIO
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [12]:
# 2. Create a temporary view of the DataFrame.
response = requests.get(url)
csv_data = response.text
# Convert the CSV data to a Pandas DataFrame
pandas_df = pd.read_csv(StringIO(csv_data))
# Convert the Pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(pandas_df)
df.createOrReplaceTempView("home_sales")
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date='2022-04-08', date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY date_built
ORDER BY date_built
"""

result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [16]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [17]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [18]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
SELECT ROUND(AVG(view), 2) AS avg_view
FROM home_sales
WHERE price >= 350000
"""

result = spark.sql(query)
# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.8104205131530762 seconds ---


In [19]:
# 7.  Cache the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
result = spark.sql("""
    SELECT AVG(view) AS average_view
    FROM home_sales
    WHERE price >= 350000
""")

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+------------+
|average_view|
+------------+
|      32.264|
+------------+

--- 3.0393455028533936 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df = spark.sql("SELECT * FROM home_sales")
home_sales_df.write.partitionBy("date_built").parquet("parquet_data")

In [27]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("parquet_data")

# Show the first few rows of the DataFrame
parquet_df.show()

+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|62c69180-ab61-47c...|2020-07-06| 395216|       4|        2|       2428|    8908|     2|         0|  20|      2017|
|273b36f4-26eb-4dd...|2021-04-27| 306223|       2|        2|       2181|   12568|     1|         0|   2|      2017|
|e2919f3b-af95-464...|2021-12-12| 152105|       2|        3|       1998|   11236|     2|         0|  21|      2017|
|1c0eeb36-0f40-479...|2022-04-07| 150817|       3|        3|       2283|    8650|     2|         0|   8|      2017|
|29a05191-0b08-4ac...|2021-03-02|1471564|       8|        3|       5088|   10228|     2|         0|  97|      2017|
|db7e9f31-e4fd-407...|2019-12-29| 152809|       3|        3|       1834|

In [28]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")
# Query temporary table to check if it's created successfully
spark.sql("SELECT * FROM parquet_home_sales").show()

+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+-------+--------+---------+-----------+--------+------+----------+----+----------+
|62c69180-ab61-47c...|2020-07-06| 395216|       4|        2|       2428|    8908|     2|         0|  20|      2017|
|273b36f4-26eb-4dd...|2021-04-27| 306223|       2|        2|       2181|   12568|     1|         0|   2|      2017|
|e2919f3b-af95-464...|2021-12-12| 152105|       2|        3|       1998|   11236|     2|         0|  21|      2017|
|1c0eeb36-0f40-479...|2022-04-07| 150817|       3|        3|       2283|    8650|     2|         0|   8|      2017|
|29a05191-0b08-4ac...|2021-03-02|1471564|       8|        3|       5088|   10228|     2|         0|  97|      2017|
|db7e9f31-e4fd-407...|2019-12-29| 152809|       3|        3|       1834|

In [29]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS avg_view
    FROM parquet_home_sales
    WHERE price >= 350000
""")

# Show the result
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 1.3188464641571045 seconds ---


In [30]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [31]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')

if not is_cached:
    print("The home_sales temporary table is no longer cached.")
else:
    print("The home_sales temporary table is still cached.")


The home_sales temporary table is no longer cached.
